<a href="https://colab.research.google.com/github/Introvert-Guy/Plagarism-Checker-AI/blob/main/Plagarism_Checker_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import zipfile
import os

# Extracting the dataset from ZIP
zip_file_path = "/content/drive/MyDrive/mit-plagiarism-detection-dataset.zip"
extract_path = "/content/mit_plagiarism_dataset"

if os.path.exists(zip_file_path):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Successfully extracted to '{extract_path}'")
    except zipfile.BadZipFile:
        print("Error: Not a valid zip file.")
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Error: ZIP file not found")


Successfully extracted to '/content/mit_plagiarism_dataset'


In [15]:
import pandas as pd

# Load the dataset
data_file = '/content/mit_plagiarism_dataset/train_snli.txt'
df = pd.read_csv(data_file, sep='\t', header=None, names=['sentence1', 'sentence2', 'label'])
print("Sample data from dataset:")
print(df.head())
print("Label distribution:")
print(df['label'].value_counts(), "\n")


Sample data from dataset:
                                           sentence1  \
0  A person on a horse jumps over a broken down a...   
1  A person on a horse jumps over a broken down a...   
2              Children smiling and waving at camera   
3              Children smiling and waving at camera   
4  A boy is jumping on skateboard in the middle o...   

                                       sentence2  label  
0  A person is at a diner, ordering an omelette.      0  
1              A person is outdoors, on a horse.      1  
2                     There are children present      1  
3                          The kids are frowning      0  
4              The boy skates down the sidewalk.      0  
Label distribution:
label
0    183966
1    183407
Name: count, dtype: int64 



In [4]:
df = df.dropna()
df['label'] = df['label'].astype(int)


<ipython-input-4-d0cb2408cdd5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


In [8]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00


In [9]:
# Loading sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings1 = model.encode(df['sentence1'].tolist(), convert_to_numpy=True, show_progress_bar=True)
embeddings2 = model.encode(df['sentence2'].tolist(), convert_to_numpy=True, show_progress_bar=True)

features = np.concatenate([
    np.abs(embeddings1 - embeddings2),
    embeddings1 * embeddings2
], axis=1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11481 [00:00<?, ?it/s]

Batches:   0%|          | 0/11481 [00:00<?, ?it/s]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, df['label'], test_size=0.2, random_state=42)


In [11]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [12]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Testing Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


Training Accuracy: 0.8375712414297624
Testing Accuracy: 0.8362005607425756

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.82      0.83     36586
           1       0.83      0.85      0.84     36888

    accuracy                           0.84     73474
   macro avg       0.84      0.84      0.84     73474
weighted avg       0.84      0.84      0.84     73474



In [19]:
import joblib
from sentence_transformers import SentenceTransformer
import numpy as np


def check_plagiarism(sentence1, sentence2):
    emb1 = model.encode([sentence1], convert_to_numpy=True)
    emb2 = model.encode([sentence2], convert_to_numpy=True)

    features = np.concatenate([np.abs(emb1 - emb2), emb1 * emb2], axis=1)
    pred = clf.predict(features)[0]
    prob = clf.predict_proba(features)[0][pred]

    if pred == 1:
        print(f"Plagiarism Detected! Similarity Confidence: {prob:.2f}")
    else:
        print(f"No Plagiarism Detected. Similarity Confidence: {prob:.2f}")

print("\nEnter two sentences to check for plagiarism similarity:")
s1 = input("Enter first sentence: ")
s2 = input("Enter second sentence: ")

check_plagiarism(s1, s2)



Enter two sentences to check for plagiarism similarity:
Enter first sentence: A man is riding a horse
Enter second sentence: A woman is cooking dinner.
No Plagiarism Detected. Similarity Confidence: 1.00
